**Connect google drive**

In [35]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Jul 31 15:35:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    86W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 15573686464772836946600196822111.jpg
 15573686722502623506381280557123.jpg
 2019-05-09-07-52-17-596.jpg
'acc details ticket.gdoc'
'accommodation n registration robocon (1).docx'
'accommodation n registration robocon (1).gdoc'
'accommodation n registration robocon (2).gdoc'
'accommodation n registration robocon.docx'
'accommodation n registration robocon.gdoc'
'advice letter_ neelesh_nmims.gdoc'
 application.mohit.docx
 application.sandeep.docx
'A Quick Guide

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14086, done.
remote: Total 14086 (delta 0), reused 0 (delta 0), pack-reused 14086
Receiving objects: 100% (14086/14086), 12.72 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (9587/9587), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [30]:
!echo "number_plate" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/train.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3\nvalid=data/train.txt' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-31 13:15:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   467KB/s    in 5m 8s   

2020-07-31 13:20:59 (516 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [19]:
!unrar e /mydrive/Yolo_V3_for_OpenCV/number_plate/images.rar -d data/obj


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /mydrive/Yolo_V3_for_OpenCV/number_plate/images.rar

Extracting  data/obj/8.txt                                                 0%  OK 
Extracting  data/obj/9.jpg                                                 0%  OK 
Extracting  data/obj/9.txt                                                 0%  OK 
Extracting  data/obj/10.jpg                                                0%  OK 
Extracting  data/obj/10.txt                                                0%  OK 
Extracting  data/obj/11.jpg                                                0%  OK 
Extracting  data/obj/11.txt                                                0%  OK 
Extracting  data/obj/12.jpg                                                0%  OK 
Extracting  data/obj/12.txt                                                0%  OK 
Extracting  data/obj/13.jpg       

In [20]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.647635 0.550571 0.137031 0.040783

['15', '0.647635', '0.550571', '0.137031', '0.040783']
0 data/obj/frame11885_leftImg8bit.txt
0 0.647635 0.550571 0.137031 0.040783
15 0.533014 0.592944 0.038278 0.027137

['15', '0.533014', '0.592944', '0.038278', '0.027137']
1 data/obj/frame3299_leftImg8bit.txt
0 0.533014 0.592944 0.038278 0.027137
15 0.147135 0.538301 0.177083 0.107242

['15', '0.147135', '0.538301', '0.177083', '0.107242']
2 data/obj/38.txt
0 0.147135 0.538301 0.177083 0.107242
15 0.530599 0.444444 0.084635 0.048611

['15', '0.530599', '0.444444', '0.084635', '0.048611']
3 data/obj/116.txt
0 0.530599 0.444444 0.084635 0.048611
15 0.416667 0.317708 0.333333 0.041667

['15', '0.416667', '0.317708', '0.333333', '0.041667']
4 data/obj/113.txt
0 0.416667 0.317708 0.333333 0.041667
15 0.423744 0.698805 0.091854 0.059727

['15', '0.423744', '0.698805', '0.091854', '0.059727']
5 data/obj/frame64499_leftImg8bit.txt
0 0.423744 0.698805 0.091854 0.059727
15 0.380208 0.545573 0.170139 0.0

In [21]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/frame59862_leftImg8bit.jpg', 'data/obj/frame0611_leftImg8bit.jpg', 'data/obj/98.jpg', 'data/obj/109.jpg', 'data/obj/40.jpg', 'data/obj/15.jpg', 'data/obj/frame1458_leftImg8bit.jpg', 'data/obj/frame27573_leftImg8bit.jpg', 'data/obj/frame3179_leftImg8bit.jpg', 'data/obj/72.jpg', 'data/obj/12.jpg', 'data/obj/frame4067_leftImg8bit.jpg', 'data/obj/152.jpg', 'data/obj/131.jpg', 'data/obj/frame18212_leftImg8bit.jpg', 'data/obj/125.jpg', 'data/obj/frame9020_leftImg8bit.jpg', 'data/obj/148.jpg', 'data/obj/145.jpg', 'data/obj/64.jpg', 'data/obj/67.jpg', 'data/obj/frame10775_leftImg8bit.jpg', 'data/obj/frame6797_leftImg8bit_1.jpg', 'data/obj/140.jpg', 'data/obj/54.jpg', 'data/obj/35.jpg', 'data/obj/frame1084_leftImg8bit.jpg', 'data/obj/frame1012_leftImg8bit.jpg', 'data/obj/111.jpg', 'data/obj/105.jpg', 'data/obj/frame23158_leftImg8bit.jpg', 'data/obj/130.jpg', 'data/obj/frame12612_leftImg8bit.jpg', 'data/obj/142.jpg', 'data/obj/frame5518_leftImg8bit.jpg', 'data/obj/60.jpg', 'data/obj/6

In [22]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [27]:
#!cp data/train.txt data/test.txt

**6) Start the training**

In [34]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -map -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.003599, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.006213, iou_loss = 0.000000, total_loss = 0.006213 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.591870, GIOU: 0.542414), Class: 0.528726, Obj: 0.002428, No Obj: 0.001825, .5R: 1.000000, .75R: 0.000000, count: 1, class_loss = 0.311406, iou_loss = 0.161183, total_loss = 0.472589 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.339033, GIOU: 0.280288), Class: 0.418242, Obj: 0.001192, No Obj: 0.000843, .5R: 0.000000, .75R: 0.000000, count: 2, class_loss = 0.676060, iou_loss = 1.100562, total_loss = 1.776623 
 total_bbox = 22963, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 